# Test of module myutil.cache
Documentation: [here](../doc/_build/html/cache.html)

In [1]:
%autoreload 2
#!rm -f cache.dir/*
#!DEL /Q cache.dir\*

## Base caching

In [2]:
import logging
logging.basicConfig(level='INFO')
from myutil.demo.cache import simplefunc as f
f.cache.clear()
f.cache.info()

BlockInfo(functor=(myutil.demo.cache.simplefunc, <Signature (x, y=3)>), hits=0, misses=0, size=0)

All cache lookups miss (if cache was cleared):

In [3]:
for i in range(15): f(1,y=i)

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> MISS(1)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> MISS(2)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> MISS(3)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> MISS(4)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> MISS(5)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> MISS(6)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> MISS(7)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> MISS(8)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> 

In [4]:
f.cache.info()

BlockInfo(functor=(myutil.demo.cache.simplefunc, <Signature (x, y=3)>), hits=0, misses=15, size=15)

In [5]:
f.cache

Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)>

All cache lookups hit, except last one (for i=15):

In [6]:
for i in range(5,10): f(y=i,x=1)
for i in range(12,16): f(1,i)

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> HIT(6)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> HIT(7)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> HIT(8)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> HIT(9)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> HIT(10)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> HIT(13)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> HIT(14)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> HIT(15)
INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> MISS

In [7]:
f.cache

Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)>

In [8]:
f.cache.clear_overflow(4)
f.cache

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)> DELETED(12)


Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.simplefunc(x, y=3)>

## Error management
Restart kernel before run.

In [1]:
import logging
logging.basicConfig(level='INFO')
from myutil.demo.cache import longfunc
longfunc.cache.clear()
print(longfunc.cache.info())
class timing:
    def __enter__(self): from time import perf_counter; self.time = perf_counter(); return self
    def __exit__(self,*a): from time import perf_counter; self.time = perf_counter()-self.time

BlockInfo(functor=(myutil.demo.cache.longfunc, <Signature (x, delay=10)>), hits=0, misses=0, size=0)


Cache lookup misses (if cache was cleared).
Execution lasts 3 seconds and raises exception, which is cached.
On second iteration, cache hits and exception is directly raised.

In [2]:
for i in range(1,3):
    print('Iteration',i,end='...',flush=True)
    with timing() as T:
        try: v = longfunc(None,3)
        except Exception as e: v = 'Exception[{}]'.format(e)
    print('result:',v,'({:.01f}sec)'.format(T.time),flush=True)

Iteration 1...

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.longfunc(x, delay=10)> MISS(17)


result: Exception[longfunc error] (3.1sec)
Iteration 2...

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.longfunc(x, delay=10)> HIT(17)


result: Exception[longfunc error] (0.0sec)


Note that cache values which are exceptions are marked with a star in the size column.

In [3]:
longfunc.cache

Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.longfunc(x, delay=10)>

Now, interrupt execution for 42,5 in iteration 1. The ``KeyboardInterrupt`` exception is cached. The ``clear_error`` call will clear that cached value so iteration 2 will complete successufully.

In [4]:
for i in range(1,3):
    for x,t in (36,2),(42,5):
        print('Iteration',i,(x,t),end='...',flush=True)
        with timing() as T:
            try: v = longfunc(x,t)
            except BaseException as e: v = 'Exception[{}]'.format(e)
        print('result:',v,'({:.01f}sec)'.format(T.time),flush=True)
    longfunc.cache.clear_error()

Iteration 1 (36, 2)...

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.longfunc(x, delay=10)> MISS(18)


result: 36 (2.1sec)
Iteration 1 (42, 5)...

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.longfunc(x, delay=10)> MISS(19)


result: Exception[] (2.3sec)


...

Iteration 2 (36, 2)...

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.longfunc(x, delay=10)> HIT(18)


result: 36 (0.0sec)
Iteration 2 (42, 5)...

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.longfunc(x, delay=10)> MISS(20)


result: 42 (5.1sec)


In [5]:
longfunc.cache

Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.longfunc(x, delay=10)>

## Processes

Restart the kernel before run.

In [1]:
import logging
logging.basicConfig(level='INFO')
from myutil.demo.cache import proc, stepA, stepB
#for s in (stepA,stepB): s.cache.clear() # un/comment to clear/keep cache from previous run
e = proc()

In [2]:
e['abc']

INFO:myutil.cache:Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.stepB(E, fr=None, to=None, r=0)> HIT(21)


14

In [3]:
stepB.cache

Cache</home/andreoli/src/PYTOOLS/demo/cache.dir:myutil.demo.cache.stepB(E, fr=None, to=None, r=0)>

## Direct cache content display

In [1]:
from myutil.cache import CacheDB
db = CacheDB('cache.dir')
db.clear_obsolete(None)
db

DEL myutil.demo.cache.vfunc{3893}(x)? 
DEL myutil.demo.cache.vfunc{3660}(x)? 
DEL myutil.demo.cache.vfunc{3930}(x)? 
DEL myutil.demo.cache.vfunc{3946}(x)? 


Cache</home/andreoli/src/PYTOOLS/demo/cache.dir>

When a cache block has its cacheonly attribute set, attempts to create new cells fail.

In [2]:
from myutil.demo.cache import simplefunc as f
f.cache.cacheonly = True
print('f(1,2)=',end='');print(f(1,12))
print('f(2)=',end='');print(f(2))

f(1,2)=(1, 12)
f(2)=

Exception: Cache cell creation disallowed